### Testing

In [1]:
import cv2
import face_recognition
import cvzone

In [70]:
img  = face_recognition.load_image_file("Train Image/Sujeet.jpg")
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

img_test = face_recognition.load_image_file("Train Image/Amit.jpg")
img_test = cv2.cvtColor(img_test, cv2.COLOR_BGR2RGB)



face_loc = face_recognition.face_locations(img)[0]
encod_face = face_recognition.face_encodings(img)[0]
cv2.rectangle(img, (face_loc[3],face_loc[0]),(face_loc[2],face_loc[1]),(255,0,255),2)

faceLocTest = face_recognition.face_locations(img_test)[0]
encodeTest  = face_recognition.face_encodings(img_test)[0]
cv2.rectangle(img_test, (faceLocTest[3],faceLocTest[0]),(faceLocTest[2],faceLocTest[1]),(255,0,255),2)

results = face_recognition.compare_faces([encod_face],encodeTest)
faceDis = face_recognition.face_distance([encod_face],encodeTest)
print(results, faceDis)

cv2.putText(img_test, f'{results} {round(faceDis[0])}',(50,50),cv2.FONT_HERSHEY_COMPLEX,1,(0,0,255),2)

cv2.imshow("Image",img)
cv2.imshow("test image", img_test)
k = cv2.waitKey(0)


[False] [0.63055531]


### Attendance Project

In [4]:
import cv2
import numpy as np
import face_recognition
import os
from datetime import datetime
from PIL import ImageGrab

path = 'Train Image'
images = []
classNames = []
myList = os.listdir(path)
#print(myList)

for cl in myList:
    curImg = cv2.imread(f'{path}/{cl}')
    images.append(curImg)
    classNames.append(os.path.splitext(cl)[0])

#print(classNames)

def findEncodings(images):
    encodeList = []
    for img in images:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        encode = face_recognition.face_encodings(img)[0]
        encodeList.append(encode)
    return encodeList

def markAttendance(name):
    with open("Attendances.csv",'r+') as f:
        myDataList = f.readlines()
        nameList = []
        
        for line in myDataList:
            entry = line.split(",")
            nameList.append(entry[0])
            
        if name not in nameList:
            now = datetime.now()
            dtString = now.strftime('%H:%M:%S')
            f.writelines(f'\n{name},{dtString}')


encodeListKnown = findEncodings(images)
print("Encoding Complete")



Encoding Complete


In [7]:
cap = cv2.VideoCapture(1)
name = "Unknown"
while True:
    success, img = cap.read()
    imgS = cv2.resize(img, (0,0), None, 0.25,0.25)
    imgS = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB)
    
    faceCurFrame = face_recognition.face_locations(imgS)
    encodeCurFrame = face_recognition.face_encodings(imgS, faceCurFrame)
    
    for encodeFace, (top, right, bottom, left) in zip(encodeCurFrame, faceCurFrame):
        matches = face_recognition.compare_faces(encodeListKnown, encodeFace)
        faceDis = face_recognition.face_distance(encodeListKnown, encodeFace)
        
        matchIndex = np.argmin(faceDis)
        
        
        
        if matches[matchIndex]:
            name = classNames[matchIndex].upper()
    
   
        # Scale back up face locations since the frame we detected in was scaled to 1/4 size
        top *= 4
        right *= 4
        bottom *= 4
        left *= 4

        # Draw a box around the face
        cv2.rectangle(img, (left+10, top+10), (right+10, bottom+10), (0, 0, 255), 2)

        # Draw a label with a name below the face
        cv2.rectangle(img, (left+10, bottom -35), (right+10, bottom+10), (0, 0, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(img, name, (left + 8, bottom - 6), font, 1.0, (255, 255, 255), 1)
        markAttendance(name)

    # Display the resulting image
    cv2.imshow('Video', img)

    # Hit 'q' on the keyboard to quit!
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release handle to the webcam
cap.release()
cv2.destroyAllWindows()
